In [ ]:
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import openai

openai.api_key = os.getenv('OPENAI_API_KEY1', 'YourAPIKeyIfNotSet')
# llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

openai.api_key = "shale-xKZC4rjZzC7XGl" 
openai.api_base = "https://shale.live/v1"
model = "longchat-13b-16k"

llm = OpenAI(model_name=model, temperature=0, openai_api_key=openai.api_key)

In [ ]:
file_path = '/Users/clzhang/Desktop/can_ali_paper.txt'
with open(file_path, 'r') as file:
    text = file.read()

# Printing the first 285 characters as a preview
print (text[:285])

num_tokens = llm.get_num_tokens(text)
print (f"There are {num_tokens} tokens in your file")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=3000, chunk_overlap=350)
docs = text_splitter.create_documents([text])

print (f"You now have {len(docs)} docs intead of 1 piece of text")

In [ ]:
chain = load_summarize_chain(llm=llm, chain_type='map_reduce')#, verbose=True) #  optional to see what is getting sent to the LLM

# Use it. This will run through the 4 documents, summarize the chunks, then get a summary of the summary.
output = chain.run(docs)
print (output)

In [69]:
from typing import Any, List, Mapping, Optional
import requests

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

class CustomLLM(LLM):
    model_engine: str
    max_tokens: int

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return self.get_response_of_modelhub(prompt)

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"model_engine": self.model_engine, "max_tokens": self.max_tokens}

    def get_response_of_modelhub(self, user_msg, system_msg=""):
        model_engine_dict = {
            "baichuan2-13b-chat-chatsft-epo5": "k8s-sv0-us1iys-1696926571411"
        }
        model_id = model_engine_dict[self.model_engine]

        url = 'http://luban-llm.intra.xiaojukeji.com/generate'
        headers = {
            'X-Luban-LLM-Service-APPId': model_id,
            'X-ModelHub-Token': '45GKEKUHEPIOWFF5Y4JVZH26UTF7ZXZMCAIGUIFOPXKK2XSX6QGCJCPGRW5ENZBTOAJW7KGK73FC7GYS6UXXZJWMW3N3F3CE3VLODP3MIQQUQ7JFTUVHROCDH7GK7EIEVBJ5SMZLXNFLNYUHMCBZTD6CQM======',
            'Content-Type': 'application/json'
        }

        msg = user_msg
        # print('msg:\n{}'.format(msg))
        data = {
            'inputs': msg,
            'parameters': {
                'max_new_tokens': self.max_tokens,
                "temperature":0.1,
                "top_p": 0.9,
                "repetition_penalty": 1.1
                }
        }
        response = requests.post(url, headers=headers, json=data)
        # print('response:\n {}'.format(response.json()))
        # print('response:\n {}'.format(response.json()['generated_text']))
        return response.json()['generated_text']


In [36]:
import json

all_chats=[]
for line in open('data/wechat/仙人指路.json'):
    chat = json.loads(line)
    msg_id, uid, type, ts, msg = chat['msg_id'], chat['uid'], chat['type'], chat['ts'], chat['msg']
    if type == 1:
        uid_msg = "{}：{}".format(uid, msg)
        all_chats.append(uid_msg)
print(len(all_chats), all_chats[:10])

[print(c) for c in all_chats[:10]]

20930 ['nameding：@🌸st*雪（阳）\u2005', 'nameding：你51个红包多少钱', 'wxid_2k71jfnx1xbp22：没看啊，没几毛', 'nameding：8 帮我助力 f:/ 吧，上抖-yin一起冲吧！理唱也拉慢名究但対', 'nameding：点了十几个红包才一块', 'wxid_2xgo1gspcjag22：注意氢能源    制氢，储氢', 'zzj457070921：兰石重装我一直埋伏着', 'wxid_2xgo1gspcjag22：镇洋已经启动   乔源也涨了', 'wxid_2xgo1gspcjag22：收获宇邦新材   鼎际得', 'wxid_2xgo1gspcjag22：这两个走的很强']
nameding：@🌸st*雪（阳） 
nameding：你51个红包多少钱
wxid_2k71jfnx1xbp22：没看啊，没几毛
nameding：8 帮我助力 f:/ 吧，上抖-yin一起冲吧！理唱也拉慢名究但対
nameding：点了十几个红包才一块
wxid_2xgo1gspcjag22：注意氢能源    制氢，储氢
zzj457070921：兰石重装我一直埋伏着
wxid_2xgo1gspcjag22：镇洋已经启动   乔源也涨了
wxid_2xgo1gspcjag22：收获宇邦新材   鼎际得
wxid_2xgo1gspcjag22：这两个走的很强


[None, None, None, None, None, None, None, None, None, None]

In [64]:
def _format_msg(system_msg, user_msg, model_engine="Baichuan2-13b-chat"):
    if model_engine == "Baichuan2-13b-chat":
        msg = "<reserved_106>{}聊天记录均在```内。```{}```<reserved_107>".format(system_msg, user_msg)
    elif model_engine == "Baichuan-13B-Chat":
        msg = "<reserved_102>{}聊天记录均在```内。```{}```<reserved_103>".format(system_msg, user_msg)
    elif model_engine == "chatglm2-6b":
        msg = "[Round 1]\n\n问：{}聊天记录均在```内。```{}```\n\n答：".format(system_msg, user_msg)
    elif model_engine == "Llama-2-13b-chat-hf":
        msg = "<<SYS>>\n{}\n<</SYS>>\n\n[INST]{}[/INST]".format(system_msg, user_msg)
    else:
        msg = "{}聊天记录均在```内。```{}```".format(system_msg, user_msg)
    return msg

system_msg = '''你是一个优秀的对话聊天助手。基于所有多轮聊天记录，你需要逐步完成如下任务，首先，将聊天记录的内容归纳出2-5个核心话题（话题名称要简洁），\
    并简洁的概括出每个人的观点或态度；最后针对每个话题给出总结和要点。提示：`<|im_start|>`代表每条聊天的开始，`<|im_end|>\n`代表每条聊天的结束。'''
user_msgs=[]
chat_group_len=80
chat_group_num=len(all_chats)//chat_group_len + 1
for i in range(chat_group_num):
    chat_group = all_chats[i*chat_group_len:(i+1)*chat_group_len]
    # random.shuffle(chat_group)
    user_msg="".join(["<|im_start|>{}<|im_end|>\n".format(chat) for chat in chat_group])
    user_msgs.append((system_msg, user_msg, ""))  # (instruction, input, output)
    # user_msgs.append((_format_msg(system_msg, user_msg), "", ""))  # (instruction, input, output)
print(len(user_msgs), len(user_msgs[0][0]), user_msgs[0])


262 155 ('你是一个优秀的对话聊天助手。基于所有多轮聊天记录，你需要逐步完成如下任务，首先，将聊天记录的内容归纳出2-5个核心话题（话题名称要简洁），    并简洁的概括出每个人的观点或态度；最后针对每个话题给出总结和要点。提示：`<|im_start|>`代表每条聊天的开始，`<|im_end|>`代表每条聊天的结束。', '<|im_start|>nameding：@🌸st*雪（阳）\u2005<|im_end|>\n<|im_start|>nameding：你51个红包多少钱<|im_end|>\n<|im_start|>wxid_2k71jfnx1xbp22：没看啊，没几毛<|im_end|>\n<|im_start|>nameding：8 帮我助力 f:/ 吧，上抖-yin一起冲吧！理唱也拉慢名究但対<|im_end|>\n<|im_start|>nameding：点了十几个红包才一块<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：注意氢能源    制氢，储氢<|im_end|>\n<|im_start|>zzj457070921：兰石重装我一直埋伏着<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：镇洋已经启动   乔源也涨了<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：收获宇邦新材   鼎际得<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：这两个走的很强<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：这算不算管不住手？<|im_end|>\n<|im_start|>nameding：买的啥<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：宇邦<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：@St*明明(投资是亏损的源头)\u2005<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：鼎际得<|im_end|>\n<|im_start|>nameding：又是次新股<|im_end|>\n<|im_start|>wxid_2xgo1gsp

In [71]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

def get_response_of_long_chat_summary(system_msg, user_msg, model_engine="baichuan2-13b-chat-chatsft-epo5", max_tokens=1000, chunk_size=1500, chunk_overlap=200):
    llm = CustomLLM(model_engine=model_engine, max_tokens=max_tokens)

    # 初始化文本分割器
    # splitter = RecursiveCharacterTextSplitter(separators=["<|im_end|>\n"], chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    splitter = RecursiveCharacterTextSplitter(separators=["\n"], chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    def long_chat_summary(llm, chat_log):
        # 使用文本分割器将聊天记录分割成多个块
        docs = splitter.create_documents([chat_log])
        print(len(docs[0].page_content), docs[0])

        map_prompt = PromptTemplate(template=_format_msg(system_msg, "{text}"), input_variables=["text"])
        combine_prompt = PromptTemplate(template="{text}", input_variables=["text"])
        chain = load_summarize_chain(llm=llm, chain_type='map_reduce', map_prompt=map_prompt, verbose=True)#, combine_prompt=combine_prompt) #
        output = chain.run(docs)
        return output

    # 测试用例
    # user_msg = ('这是一段非常长的聊天记录，需要被分割成多个块进行摘要.' * 5 + '<|im_end|>\n') *5
    summaries = long_chat_summary(llm, user_msg)
    return summaries

# get_response_of_long_chat_summary(system_msg,"")

In [70]:
model_engines=["baichuan2-13b-chat-chatsft-epo5"]

resps_model = []
for inst,input,gpt_output in user_msgs[:1]:
    resps=[inst,gpt_output]
    for model in model_engines:
        try:
            # response = get_response_of_modelhub(inst, system_msg=input, model_engine=model, max_tokens=1000)
            response = get_response_of_long_chat_summary(inst, input, model_engine=model, max_tokens=1000)
        except requests.exceptions.ConnectionError as e:
            print(f"{e}")
            # time.sleep(3)
            continue
        # except Exception as e:
        #     print(f"捕获到异常：{e}")
        #     continue
        resps.append(response)
        print("\n# {}\n===========================================".format("gpt-35-turbo"))
        print(gpt_output)
        print("\n# {}\n===========================================".format(model))
        # print(response['generated_text'])
        print(response)
        print('\n\n')
    resps_model.append(resps)

1473 page_content='<|im_start|>nameding：@🌸st*雪（阳）\u2005<|im_end|>\n<|im_start|>nameding：你51个红包多少钱<|im_end|>\n<|im_start|>wxid_2k71jfnx1xbp22：没看啊，没几毛<|im_end|>\n<|im_start|>nameding：8 帮我助力 f:/ 吧，上抖-yin一起冲吧！理唱也拉慢名究但対<|im_end|>\n<|im_start|>nameding：点了十几个红包才一块<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：注意氢能源    制氢，储氢<|im_end|>\n<|im_start|>zzj457070921：兰石重装我一直埋伏着<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：镇洋已经启动   乔源也涨了<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：收获宇邦新材   鼎际得<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：这两个走的很强<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：这算不算管不住手？<|im_end|>\n<|im_start|>nameding：买的啥<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：宇邦<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：@St*明明(投资是亏损的源头)\u2005<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：鼎际得<|im_end|>\n<|im_start|>nameding：又是次新股<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：对<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：这种屡创新高的个股   我还是比较看好   咬一口就走了<|im_end|>\n<|im_start|>wxid_2xgo1gspcjag22：@St*明明(投资是亏损的源头)\u2005收盘能红

Token indices sequence length is longer than the specified maximum sequence length for this model (4578 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1926 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3062 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2671 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2148 > 1024). Running this sequence through the model will result in indexing errors



# gpt-35-turbo


# baichuan2-13b-chat-chatsft-epo5
话题1：微信红包金额
- nameding的观点：觉得花8块钱买51个红包很不划算。
- wxid_2k71jfnx1xbp22的观点：对微信红包的金额不感兴趣，认为没有几毛钱。

话题2：股票投资策略
- wxid_2xgo1gspcjag22的观点：关注氢能源、制氢、储氢等概念股，认为镇洋和乔源走势强劲。
- zzj457070921的观点：潜伏在兰石重装这只股票中，认为该股有望上涨。

话题3：新股交易策略
- wxid_2xgo1gspcjag22的观点：对新股鼎际得和宇邦新材持乐观态度，认为它们走势强劲。
- nameding的观点：认同鼎际得的强势表现，但对宇邦新材持有怀疑态度。

总结和要点：
- 微信红包金额：nameding觉得花8块钱买51个红包很不划算，而wxid_2k71jfnx1xbp22对微信红包的金额不感兴趣。
- 股票投资策略：wxid_2xgo1gspcjag22关注氢能源、制氢、储氢等概念股，认为镇洋和乔源走势强劲，而zzj457070921潜伏在兰石重装这只股票中。
- 新股交易策略：wxid_2xgo1gspcjag22对新股鼎际得和宇邦新材持乐观态度，认为它们走势强劲，而nameding对宇邦新材持有怀疑态度。



